# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f014ce19430>
├── 'a' --> tensor([[-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247]])
└── 'x' --> <FastTreeValue 0x7f014ce13370>
    └── 'c' --> tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                        [-0.5879,  0.0509, -1.1617, -0.1679],
                        [-1.1565,  1.0338, -0.1534,  0.5944]])

In [4]:
t.a

tensor([[-1.2060, -0.8508, -1.7793],
        [ 1.5509, -0.4654,  1.5247]])

In [5]:
%timeit t.a

63.7 ns ± 0.0637 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f014ce19430>
├── 'a' --> tensor([[ 1.1814,  1.0716,  0.8646],
│                   [-1.6493, -0.3131,  1.3422]])
└── 'x' --> <FastTreeValue 0x7f014ce13370>
    └── 'c' --> tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                        [-0.5879,  0.0509, -1.1617, -0.1679],
                        [-1.1565,  1.0338, -0.1534,  0.5944]])

In [7]:
%timeit t.a = new_value

66.1 ns ± 0.0314 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247]]),
    x: Batch(
           c: tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                      [-0.5879,  0.0509, -1.1617, -0.1679],
                      [-1.1565,  1.0338, -0.1534,  0.5944]]),
       ),
)

In [10]:
b.a

tensor([[-1.2060, -0.8508, -1.7793],
        [ 1.5509, -0.4654,  1.5247]])

In [11]:
%timeit b.a

64.3 ns ± 0.00949 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0733, -2.0277,  0.8680],
               [ 0.6840,  0.6269, -0.6258]]),
    x: Batch(
           c: tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                      [-0.5879,  0.0509, -1.1617, -0.1679],
                      [-1.1565,  1.0338, -0.1534,  0.5944]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

885 ns ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 667 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 541 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f014ce206d0>
├── 'a' --> tensor([[[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]],
│           
│                   [[-1.2060, -0.8508, -1.7793],
│                    [ 1.5509, -0.4654,  1.5247]]])
└── 'x' --> <FastTreeValue 0x7f014ce20bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 33.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f00a1ad56a0>
├── 'a' --> tensor([[-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247],
│                   [-1.2060, -0.8508, -1.7793],
│                   [ 1.5509, -0.4654,  1.5247]])
└── 'x' --> <FastTreeValue 0x7f00a1ad5490>
    └── 'c' --> tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                        [-0.5879,  0.0509, -1.1617, -0.1679],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 33.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.1 µs ± 44 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]],
       
               [[-1.2060, -0.8508, -1.7793],
                [ 1.5509, -0.4654,  1.5247]]]),
    x: Batch(
           c: tensor([[[ 0.7781,  0.2077,  0.1846,  0.6649],
                       [-0.5879,  0.0509, -1.1617, -0.1679],
                       [-1.1565,  1.0338, -0.1534,  0.5944]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 251 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247],
               [-1.2060, -0.8508, -1.7793],
               [ 1.5509, -0.4654,  1.5247]]),
    x: Batch(
           c: tensor([[ 0.7781,  0.2077,  0.1846,  0.6649],
                      [-0.5879,  0.0509, -1.1617, -0.1679],
                      [-1.1565,  1.0338, -0.1534,  0.5944],
                      [ 0.7781,  0.2077,  0.1846,  0.6649],
                      [-0.5879,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 406 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
